# Gaia Sprint 2018: Making a dust attenuated SED from scratch.

## A little background

The physics of a star's intrinsic SED can be described by a combination of stellar atmosphere and stellar evolutionary models.

Additionally, we must also account for the fact that as photons travel from the star to the observer, some photons will be absorbed by the dust along the line of sight. This not only dims the star light similarly to distance but also changes its apparent color, making the star appear redder. The wavelength-dependence of the extinction from the ultra-violet (UV) to the near infrared (NIR) has been studied and modeled in multiple galaxies.  

The observations show a wide range of dust column normalized extinction curves, which are commonly characterized as a wavelength dependent flux attenuation

\begin{equation}
  \tau_\lambda = A(\lambda)/A_0 = f(\lambda; A_0, R_0),
\end{equation}

Finally, the apparent wavelength dependent light observed from a star at distance $r$ sets as
\begin{equation}
  l_{\lambda}(\theta) = \frac{L_{bol}(\theta)}{4\pi
  r^2}\,S_{\lambda}(\theta)\, e^{-0.4\,A_0\,\tau_\lambda},
  \label{eq:modelsed}
\end{equation}
where $\theta$ represents the astrophysical parameters of that star.

## requirements:

* some **isochrones**, e.g.
    * _PARSEC_ ([ezpadova](https://github.com/mfouesneau/ezpadova)), 
    * _MESA/MIST_ ([ezmist](https://github.com/mfouesneau/ezmist)), 
    * _Dartmouth_ ([ezdart](https://github.com/mfouesneau/ezdart))

* some **spectral libraries**, e.g. [pystellibs](https://github.com/mfouesneau/pystellibs), 
    * _BaSeL_: BaSeL 2.2, ~ Atlas 9 empirically recalibrated (Leujeune et al 1998)
    * _Rauch_: a White dwarf library
    * _Kurucz_: Castelli and Kurucz 2004 or ATLAS9
    * _Tlusty_: NLTE O, B stars [Lanz, T., & Hubeny, I. (2003)]
    * _Elodie_: version 3.1, high resolution optical library.
    * _Munari_: extended ATLAS9 stellar atmospheres (Munari et al. 2005 A&A 442 1127)
    * _BTSettl_: BT-Settl Library (Allard, Hauschildt and Schweitzer 2000)
    
* some **dust attenuation curves**, e.g. [pyextinction](https://github.com/mfouesneau/pyextinction)
    * _Cardelli_, Cardelli, Clayton, and Mathis (1989, ApJ, 345, 245)
    * _Calzetti_, Calzetti et al. (2000, ApJ 533, 682)
    * _Fitzpatrick_, Fitzpatrick (1999, PASP, 111, 63)
    * _Gordon03 SMCBar_, Gordon et al. 2003 (ApJ, 594:279-293)

* some passbands and tools to compute **photometry**, e.g. [pyphot](https://github.com/mfouesneau/pyphot)
    * 243 passbands and 49 spectral indices definitions
    * provides _nominal_ (Jordi+2010), _GDR2_ and _revised_ (Evans+2018), and _Weiler_ (Weiler 2018) Gaia passbands.
    * can handle custom user provided passbands.

## notebook configuration

In [1]:
# Loading configuration
# Don't forget that mac has this annoying configuration that leads
# to limited number of figures/files
# ulimit -n 4096    <---- osx limits to 256

# Notebook matplotlib mode
%pylab inline                                 
# set for retina or hi-resolution displays
%config InlineBackend.figure_format='retina'  

Populating the interactive namespace from numpy and matplotlib


## imports

In [2]:
import pylab as plt
import numpy as np
from ezdata import SimpleTable

import ezmist
import pyextinction
import pystellibs
import pyphot
from pyphot import unit   # these are NOT astropy

## Get stellar parameter grid

In this example, we download the standard set of isochrones from MESA/MIST. It is basically a solar metallicity grid of isochrones.
We filter out pre-MS, TPAGB, and WD phase to match the reliability of the spectral models we use below. 
(one can combine spectral libraries to include all the phases).

In [3]:
iso = ezmist.get_standard_isochrone()
iso = iso.selectWhere('*', '(phase >= 0) & (phase < 6)')  # get rid of PMS and TP-AGB & WD
Z = np.array([0.0145] * len(iso))
iso.add_column("Z", Z, dtype=float, description='metallicity')
iso.data['Z'] = 0.0145   # sometimes there is a bug... this should not be necessary.
iso.set_alias('logT', 'log_Teff')
iso.set_alias('logL', 'log_L')

Interrogating http://waps.cfa.harvard.edu/MIST/iso_form.php...
Request...done.
Reading content...done.
b'PK\x03\x04\x14\x00\x00\x00\x08\x00<@\xb9L:ZbTr\xd0L\x01"`\xde\x04\x1a\x00\x00\x00MIST_iso_5b07fb32283f2.iso\xbc\xfd[\x8f\x05\xd9u\xa5\x87\xbe\xebW\x14\xd0\x8f\xc7\x95\xbd\xee\x97\x87~\xf0\x83\x8c6`\x03F[/\xf6\xc1\x01QM\x97D\x02\x12\xcb \xd5\r' <class 'bytes'> b'PK\x03\x04\x14\x00\x00\x00\x08\x00'
decompressing archive (type=zip)...done.


## Generate interpolated spectral grid

For every star in our astrophysical parameter grid (isochrone set defined above), we need to compute an associated spectrum that covers the wavelength range of interest.
Below I use `BaSeL` but any other library will work and even combined with others (e.g. `lib = BaSeL() + Rauch()`)

In [4]:
lib = pystellibs.BaSeL()
# check coverage
points = np.array([iso['logT'], iso['logg']])
select = lib.points_inside(points.T)
# make the right names: logT, logg, logL, Z, must exist
# SimpleTable offers aliases
pars = SimpleTable(iso)   ## just to make a clear name
pars.data = pars.data[select]
pars.set_alias('logT', 'log_Teff')
pars.set_alias('logL', 'log_L')
pars.set_alias('logg', 'log_g')
pars.set_alias('mass', 'initial_mass')
pars.set_alias('logA', 'log10_isochrone_age_yr')
wave, specs = lib.generate_individual_spectra(pars)

## Get some photometric passbands

At this step one can define their own photometric interest. Filters will be generated and adapted to the spectral sampling of our grid, and zeropoints will be provided as well to calibrate the photometry.

Below I request GALEX, GAIA, JOHNSON, SDSS, 2MASS and WISE passbands. (to find the names in the library one can use `lib.find`)

In [5]:
import itertools

# select phot. filters
galex_names = 'GALEX_FUV GALEX_NUV'.split()
gaia_names = ['GaiaDR2_BP', 'GaiaDR2_G', 'GaiaDR2_RP']
johnson_names = ('GROUND_JOHNSON_U GROUND_JOHNSON_B GROUND_JOHNSON_V'.split()
                 + 'GROUND_COUSINS_R GROUND_COUSINS_I'.split())
sdss_names = 'SDSS_u SDSS_g SDSS_r SDSS_i SDSS_z'.split()
twomass_names = '2MASS_J 2MASS_H 2MASS_Ks'.split()
wise_names = 'WISE_RSR_W1 WISE_RSR_W2'.split()

names = tuple(itertools.chain(galex_names, gaia_names, johnson_names, sdss_names,
                              twomass_names, wise_names))

with pyphot.get_library() as lib:
    flist = lib.load_filters(names, lamb=wave)

# Compute the magnitude zero points ==============
# some surveys uses AB mag others Vega
use_AB = 'GALEX', 'SDSS'

zpts = []
for fk in flist:
    if fk.name.split('_')[0] in use_AB:
        zpts.append(fk.AB_zero_mag)
    else:
        zpts.append(fk.Vega_zero_mag)

zpts = np.array(zpts)

for a short summary, I print below the photometric parameters

In [6]:
table_rows = [("name", "zpt", "mag", "system")]

print("    Photometric magnitude zeropoints\n")
print("    {0:18s} {1:7s} {2:7s} {3:7s} ".format("name", "zpt", "mag", "system"))
print("   ", "-" * 18, "-" * 7, "-" * 7, "-" * 7)
for fk, val in zip(flist, zpts):
    if fk.name.split('_')[0] in use_AB:
        table_rows.append((fk.name, val, "ABmag", fk.dtype))
        print("    {0:18s} {1:0.4f} {2:7s} {3:s} ".format(fk.name, val, "ABmag", fk.dtype))
    else:
        table_rows.append((fk.name, val, "Vegamag", fk.dtype))
        print("    {0:18s} {1:0.4f} {2:7s} {3:s}".format(fk.name, val, "Vegamag", fk.dtype))
print("   ", "-" * 18, "-" * 7, "-" * 7, "-" * 7)

    Photometric magnitude zeropoints

    name               zpt     mag     system  
    ------------------ ------- ------- -------
    GALEX_FUV          18.3409 ABmag   photon 
    GALEX_NUV          19.2156 ABmag   photon 
    GaiaDR2_BP         20.9817 Vegamag photon
    GaiaDR2_G          21.5101 Vegamag photon
    GaiaDR2_RP         22.2198 Vegamag photon
    GROUND_JOHNSON_U   20.9460 Vegamag energy
    GROUND_JOHNSON_B   20.5122 Vegamag energy
    GROUND_JOHNSON_V   21.0991 Vegamag energy
    GROUND_COUSINS_R   21.6464 Vegamag energy
    GROUND_COUSINS_I   22.3067 Vegamag energy
    SDSS_u             20.1631 ABmag   photon 
    SDSS_g             20.7696 ABmag   photon 
    SDSS_r             21.3613 ABmag   photon 
    SDSS_i             21.7803 ABmag   photon 
    SDSS_z             22.1663 ABmag   photon 
    2MASS_J            23.7661 Vegamag photon
    2MASS_H            24.8611 Vegamag photon
    2MASS_Ks           25.9214 Vegamag photon
    WISE_RSR_W1        27.7104 V

## Compute dust attenuated spectra & photometry

At this stage, we have the APs, the spectra and the passbands. We need to first apply a dust attenuation curve on the spectra and then apply the photometric filters.

### Apply dust to the spectra

In [7]:
# select extinction
l = pyextinction.Fitzpatrick99()
A0 = 3     ## magnitudes
R0 = 3.1   ## unitless

N = len(pars)
## split values and units
_specs = specs.magnitude        
_unit = unit[str(specs.units)]  
Dlambda = np.exp(-1 * l.function(wave, Av=A0, Rv=R0))
_s = _specs * Dlambda[None, :]

# export new parameter (making a copy)
_p = SimpleTable(pars, copy=True)
_p.add_column('A0', np.ones(N) * A0, unit='mag', description='Extinction')
dust_specs = _s * _unit
dust_pars = _p

### Compute photometry

The following will integrate the flux of the spectra through many passbands (defined above) and calibrate them to be absolute magnitudes (i.e. at 10 pc).
The zeropoints contain the information on which passband uses AB system.

In [8]:
_, fluxes = pyphot.extractSEDs(wave, dust_specs.magnitude, flist, absFlux=True, progress=True)
mags = -2.5 * np.log10(fluxes) - zpts

Photometry:|##########| 20/20 100% [time: 01:00, eta: 00:00,  0.33 iters/sec]  
